Word Embedding chapter 2.1 assignment


In [29]:
with open('corpus/the-verdict.txt', 'r', encoding='utf-8') as file:
    raw_text = file.read()
print("Total number of characters in the file:", len(raw_text))
print(raw_text[:100])









Total number of characters in the file: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g


In [30]:
import re 
text='hello, world! a;lkjf i know htat if happens'
result=re.split(r'(\s)',text)
print(result)









['hello,', ' ', 'world!', ' ', 'a;lkjf', ' ', 'i', ' ', 'know', ' ', 'htat', ' ', 'if', ' ', 'happens']


In [31]:
import re 
text='hello, world! a;lkjf i know htat if happens'
result2=re.split(r'([\s,;])',text)
print(result2)









['hello', ',', '', ' ', 'world!', ' ', 'a', ';', 'lkjf', ' ', 'i', ' ', 'know', ' ', 'htat', ' ', 'if', ' ', 'happens']


In [32]:
import re

# Read the text from the file
with open('corpus/the-verdict.txt', 'r', encoding='utf-8') as f:
    verdict_text = f.read()

# Use regex to split so that every word, whitespace, and special character is a separate token
# This will match words, whitespace, or any single non-whitespace, non-word character
tokens = re.findall(r'\w+|\s+|[^\w\s]', verdict_text)

# Print the first 50 tokens as a check
print(tokens[:500])



['I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ', 'Gisburn', ' ', 'rather', ' ', 'a', ' ', 'cheap', ' ', 'genius', '-', '-', 'though', ' ', 'a', ' ', 'good', ' ', 'fellow', ' ', 'enough', '-', '-', 'so', ' ', 'it', ' ', 'was', ' ', 'no', ' ', 'great', ' ', 'surprise', ' ', 'to', ' ', 'me', ' ', 'to', ' ', 'hear', ' ', 'that', ',', ' ', 'in', ' ', 'the', ' ', 'height', ' ', 'of', ' ', 'his', ' ', 'glory', ',', ' ', 'he', ' ', 'had', ' ', 'dropped', ' ', 'his', ' ', 'painting', ',', ' ', 'married', ' ', 'a', ' ', 'rich', ' ', 'widow', ',', ' ', 'and', ' ', 'established', ' ', 'himself', ' ', 'in', ' ', 'a', ' ', 'villa', ' ', 'on', ' ', 'the', ' ', 'Riviera', '.', ' ', '(', 'Though', ' ', 'I', ' ', 'rather', ' ', 'thought', ' ', 'it', ' ', 'would', ' ', 'have', ' ', 'been', ' ', 'Rome', ' ', 'or', ' ', 'Florence', '.', ')', '\n\n', '"', 'The', ' ', 'height', ' ', 'of', ' ', 'his', ' ', 'glory', '"', '-', '-', 'that', ' ', 'was', ' ', 'what', ' ', 'the', ' ', 'women', ' '

In [33]:
tokens_no_whitespace = [token for token in tokens if not token.isspace()]
print(tokens_no_whitespace[:500])


['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '-', '-', 'though', 'a', 'good', 'fellow', 'enough', '-', '-', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow', ',', 'and', 'established', 'himself', 'in', 'a', 'villa', 'on', 'the', 'Riviera', '.', '(', 'Though', 'I', 'rather', 'thought', 'it', 'would', 'have', 'been', 'Rome', 'or', 'Florence', '.', ')', '"', 'The', 'height', 'of', 'his', 'glory', '"', '-', '-', 'that', 'was', 'what', 'the', 'women', 'called', 'it', '.', 'I', 'can', 'hear', 'Mrs', '.', 'Gideon', 'Thwing', '-', '-', 'his', 'last', 'Chicago', 'sitter', '-', '-', 'deploring', 'his', 'unaccountable', 'abdication', '.', '"', 'Of', 'course', 'it', "'", 's', 'going', 'to', 'send', 'the', 'value', 'of', 'my', 'picture', "'", 'way', 'up', ';', 'but', 'I', 'don', "'", 't', 'thin

In [34]:
len(tokens_no_whitespace)

4827

In [35]:
all_words=sorted(set(tokens_no_whitespace))
vocab_size=len(all_words)
print(f"Vocabulary size: {vocab_size}")

Vocabulary size: 1148


In [36]:
word_to_index = {word: idx for idx, word in enumerate(all_words)}
print(list(word_to_index.items())[:50])


[('!', 0), ('"', 1), ("'", 2), ('(', 3), (')', 4), (',', 5), ('-', 6), ('.', 7), (':', 8), (';', 9), ('?', 10), ('A', 11), ('Ah', 12), ('Among', 13), ('And', 14), ('Are', 15), ('Arrt', 16), ('As', 17), ('At', 18), ('Be', 19), ('Begin', 20), ('Burlington', 21), ('But', 22), ('By', 23), ('Carlo', 24), ('Chicago', 25), ('Claude', 26), ('Come', 27), ('Croft', 28), ('Destroyed', 29), ('Devonshire', 30), ('Don', 31), ('Dubarry_', 32), ('Emperors', 33), ('Florence', 34), ('For', 35), ('Gallery', 36), ('Gideon', 37), ('Gisburn', 38), ('Gisburns', 39), ('Grafton', 40), ('Greek', 41), ('Grindle', 42), ('Grindles', 43), ('HAD', 44), ('Had', 45), ('Hang', 46), ('Has', 47), ('He', 48), ('Her', 49)]


In [37]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_idx = {word: idx for idx, word in enumerate(vocab)}
        self.idx_to_str = {idx: word for word, idx in self.str_to_idx.items()}
        self.unk_token = "<unk>" if "<unk>" in self.str_to_idx else None

    def encode(self, text):
        preprocessed_text = re.sub(r'[^\w\s]', '', text)
        words = preprocessed_text.split()
        if self.unk_token is not None:
            return [self.str_to_idx[word] if word in self.str_to_idx else self.str_to_idx[self.unk_token] for word in words]
        else:
            return [self.str_to_idx[word] for word in words]

    def decode(self, ids):
        text = ' '.join([self.idx_to_str[idx] if idx in self.idx_to_str else (self.unk_token if self.unk_token is not None else '') for idx in ids])
        text = re.sub(r'\s+', ' ', text)
        return text.strip()


In [38]:
# Let's test the SimpleTokenizerV1 class with a sample text
# Ensure the sample text only contains words from the vocab
sample_text = "I had always thought Jack Gisburn rather a cheap good fellow enough so it was no great surprise to me to hear that in the height of his glory he had dropped his painting married a rich widow and established himself in a villa on the Riviera"
tokenizer = SimpleTokenizerV1(all_words)

# Encode the sample text
encoded = tokenizer.encode(sample_text)
print("Encoded:", encoded)

# Decode the encoded ids
decoded = tokenizer.decode(encoded)
print("Decoded:", decoded)


Encoded: [53, 530, 163, 1021, 57, 38, 835, 129, 271, 518, 452, 410, 925, 599, 1096, 723, 524, 979, 1034, 676, 1034, 548, 1005, 582, 1006, 551, 736, 562, 514, 546, 530, 388, 562, 762, 674, 129, 858, 1120, 170, 415, 560, 582, 129, 1085, 741, 1006, 84]
Decoded: I had always thought Jack Gisburn rather a cheap good fellow enough so it was no great surprise to me to hear that in the height of his glory he had dropped his painting married a rich widow and established himself in a villa on the Riviera


In [39]:
# Add special tokens to the vocabulary and update the mappings
special_tokens = ['<eot>', '<unk>']
for token in special_tokens:
    if token not in all_words:
        all_words.append(token)

# Update vocab_size
vocab_size = len(all_words)
print(f"Vocabulary size (with special tokens): {vocab_size}")



# Show the last few entries to confirm special tokens are present
print("Last 5 vocab entries:", list(word_to_index.items())[-5:])


Vocabulary size (with special tokens): 1150
Last 5 vocab entries: [('yet', 1143), ('you', 1144), ('younger', 1145), ('your', 1146), ('yourself', 1147)]


In [40]:
text1='hello, world! a;lkjf i know htat if happens'
text2="In the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera."

text="<eot>".join([text1,text2])
print(text)











hello, world! a;lkjf i know htat if happens<eot>In the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera.


In [41]:
tokenizer=SimpleTokenizerV1(all_words)

print(tokenizer.encode(text))










[1149, 1149, 1149, 1149, 609, 1149, 580, 1149, 1006, 551, 736, 562, 514, 546, 530, 388, 562, 762, 674, 129, 858, 1120, 170, 415, 560, 582, 129, 1085, 741, 1006, 84]


In [42]:
!pip install tiktoken


In [43]:
import tiktoken

# Common choices:
# - "cl100k_base": used by GPT-4/3.5 Turbo
# - "gpt2": used by GPT‑2/3 legacy
enc = tiktoken.get_encoding("cl100k_base")

text = "Hello, world! The Verdict by Edith Wharton."
ids = enc.encode(text)                 # list of token ids
print(ids, len(ids))
print(enc.decode(ids))                 # back to string

[9906, 11, 1917, 0, 578, 6383, 8644, 555, 3279, 411, 1254, 80596, 13] 13
Hello, world! The Verdict by Edith Wharton.


In [44]:
decoded_text = enc.decode(ids)
print(decoded_text)


Hello, world! The Verdict by Edith Wharton.


In [45]:
with open('corpus/the-verdict.txt', 'r') as file:
    raw_text = file.read()


enc_text=enc.encode(raw_text)
print(len(enc_text))




4943


In [46]:
sample_100 = enc_text[:100]
print(sample_100)


[40, 473, 1846, 2744, 3463, 7762, 480, 285, 22464, 4856, 264, 12136, 35201, 313, 4636, 264, 1695, 12637, 3403, 313, 708, 433, 574, 912, 2294, 13051, 311, 757, 311, 6865, 430, 11, 304, 279, 2673, 315, 813, 27025, 11, 568, 1047, 12504, 813, 19354, 11, 12502, 264, 9257, 57896, 11, 323, 9749, 5678, 304, 264, 47625, 389, 279, 51768, 26919, 13, 320, 27831, 358, 4856, 3463, 433, 1053, 617, 1027, 22463, 477, 48606, 9456, 10227, 2673, 315, 813, 27025, 75857, 9210, 574, 1148, 279, 3278, 2663, 433, 13, 358, 649, 6865, 18083, 13, 480, 100242, 666, 24510, 313, 26301, 1566]


In [47]:
context_size=4

x=enc_text[:context_size]


In [48]:
import torch
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)  #1

        for i in range(0, len(token_ids) - max_length, stride):  #2
            input_chunk = token_ids[i : i + max_length]
            target_chunk = token_ids[i + 1 : i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):  #3
        return len(self.input_ids)

    def __getitem__(self, idx):  #4
        return self.input_ids[idx], self.target_ids[idx]
        

In [49]:
dataset=GPTDatasetV1(txt=raw_text,tokenizer=enc,max_length=4,stride=10)

In [50]:
len(dataset)




494

In [51]:
import torch
import torch.nn as nn

# Define a simple embedding layer
vocab_size =50257 # assuming enc_text contains all token ids
embedding_dim = 256  # you can choose any dimension

embedding_layer = nn.Embedding(vocab_size, embedding_dim)

# Example: get embeddings for the first 10 tokens
input_tokens = torch.tensor(enc_text[:10])
embeddings = embedding_layer(input_tokens)
print(embeddings)


tensor([[ 1.9500, -1.7653,  0.2416,  ...,  0.4054,  0.0727,  0.2997],
        [ 0.7126, -0.0699,  0.8247,  ...,  2.9777, -0.0623, -0.0431],
        [-0.2339,  0.6607, -0.6359,  ..., -2.4909, -1.4999,  0.5544],
        ...,
        [ 2.0681, -0.7444, -1.5652,  ..., -1.4711, -1.1062, -0.5722],
        [-0.3066,  1.3249,  0.8148,  ..., -0.0111,  1.8463, -1.9940],
        [-0.3062,  0.4959,  2.7274,  ..., -1.1380, -1.1436, -0.9757]],
       grad_fn=<EmbeddingBackward0>)


In [52]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)


Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [53]:
token_embeddings=embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


absolute embeddings layer

In [54]:
context_length=max_length
pos_embedding_layer=torch.nn.Embedding(context_length,embedding_dim)
positional_embeddings=pos_embedding_layer(torch.arange(context_length))
print(positional_embeddings)

tensor([[-0.7807,  1.2045,  0.6253,  ..., -1.4217, -0.5984,  0.2741],
        [-0.3190, -0.2972, -0.6952,  ..., -0.0796,  1.1315,  0.9261],
        [ 0.1182, -0.2012,  0.6529,  ..., -0.2330, -1.2454, -0.6015],
        [-0.0603,  2.1070,  0.4009,  ...,  0.3823,  1.2538,  1.4545]],
       grad_fn=<EmbeddingBackward0>)


In [55]:
input_embeddings=positional_embeddings+token_embeddings

print(input_embeddings.shape)

torch.Size([8, 4, 256])


In [56]:
query = input_embeddings.shape
attention_scores = torch.empty(input_embeddings.shape[1])

for i, x_i in enumerate(input_embeddings[0]):
    attention_scores[i] = torch.dot(x_i, query)

print(attention_scores)


TypeError: dot(): argument 'tensor' (position 2) must be Tensor, not torch.Size

In [ ]:
query = torch.empty(input_embeddings.shape)
query.shape
attention_table_col=input_embeddings.shape[1]
attention_table_row=input_embeddings.shape[1]
attention_scores = torch.empty(8,attention_table_row,attention_table_col)
attention_weights = torch.empty(8,attention_table_row,attention_table_col)


for i,element in enumerate(input_embeddings):
    attention_scores[i]=element@element.T
    attention_weights[i]=torch.softmax(attention_scores[i],dim=1)
    # print(attention_weights)
    print(attention_weights[i].sum(dim=1))
    print(attention_weights[i].shape)
    # print(attention_weights.shape)
    query[i]=attention_weights[i]@element
    




tensor([1., 1., 1., 1.], grad_fn=<SumBackward1>)
torch.Size([4, 4])
tensor([1., 1., 1., 1.], grad_fn=<SumBackward1>)
torch.Size([4, 4])
tensor([1., 1., 1., 1.], grad_fn=<SumBackward1>)
torch.Size([4, 4])
tensor([1., 1., 1., 1.], grad_fn=<SumBackward1>)
torch.Size([4, 4])
tensor([1., 1., 1., 1.], grad_fn=<SumBackward1>)
torch.Size([4, 4])
tensor([1., 1., 1., 1.], grad_fn=<SumBackward1>)
torch.Size([4, 4])
tensor([1., 1., 1., 1.], grad_fn=<SumBackward1>)
torch.Size([4, 4])
tensor([1., 1., 1., 1.], grad_fn=<SumBackward1>)
torch.Size([4, 4])


In [ ]:
torch.manual_seed(72)
d_in=256
d_out=72
W_query=torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
W_key=torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
W_value=torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)




In [ ]:
input_vector = input_embeddings[0][0]
print(input_vector.reshape(1, -1))

tensor([[ 0.1470,  1.2117,  1.5106,  0.1727, -1.7179,  0.4756,  1.8485, -1.7623,
          1.3743, -1.6952, -0.0268, -0.1439,  0.2167, -1.4866, -1.2040,  1.1210,
          0.2711, -0.5032,  0.0782, -0.6240, -0.0831, -1.7659,  1.2772,  0.5687,
          0.4793, -2.4864, -2.7124,  0.5943,  0.9928, -0.0408, -1.5472, -0.2541,
         -0.1865, -1.5792,  0.1732, -0.5570, -2.6253, -0.1595,  0.0720,  0.8394,
          1.0381, -1.6998,  0.8757, -0.6407, -0.1187, -0.2671,  1.0992, -1.6783,
          0.6972,  1.7765, -3.7969,  1.7232,  1.7697,  1.3824, -1.8205, -1.6900,
         -0.4265,  2.5067, -0.1613, -0.6574,  0.7093, -1.9171,  2.9176, -1.5214,
          0.7911,  0.2906,  0.0747, -1.9264, -0.8710, -0.7876, -0.0451, -2.1764,
          1.3006,  0.1634,  0.5425,  1.4880,  0.2571, -0.2933,  2.0370, -0.2458,
         -0.1198,  0.5864,  0.3361,  1.4519,  1.1175, -0.0722, -2.1076,  0.0814,
          0.8313, -1.0475, -0.8123, -0.0352, -2.5903, -0.2906, -2.1194, -0.0498,
          0.1742, -0.2057, -

In [ ]:
input_to_query=input_vector@W_query
input_to_key=input_vector@W_key
input_to_value=input_vector@W_value


In [ ]:
print(input_to_query.shape)

torch.Size([72])


In [ ]:
inputs_to_querys=input_embeddings@W_query
inputs_to_keys=input_embeddings@W_key
inputs_to_values=input_embeddings@W_value

In [ ]:
print(inputs_to_querys.shape)
print(inputs_to_keys.shape)

torch.Size([8, 4, 72])
torch.Size([8, 4, 72])


In [ ]:
query_key_attention_scores=inputs_to_querys@inputs_to_keys.transpose(1,2)

In [ ]:

d_k=inputs_to_keys.shape[-1]
print(d_k)
query_key_attention_scores_normalized=torch.softmax(query_key_attention_scores/d_k**0.5,dim=-1)

72


In [ ]:
context_vector=query_key_attention_scores_normalized@inputs_to_values

In [ ]:
print(context_vector.shape)

torch.Size([8, 4, 72])


In [57]:
import torch.nn as nn
class SelfAttention_v1(nn.Module):
    def __init__(self,d_in,d_out):
        super.__init__()
        self.W_query=torch.nn.Parameter(torch.rand(d_in,d_out))
        self.W_key=torch.nn.Parameter(torch.rand(d_in,d_out))
        self.W_value=torch.nn.Parameter(torch.rand(d_in,d_out))
        
        
    def forward(self,x):
        queries=x@self.W_query
        keys=x@self.W_key
        values=x@self.W_values
        
        attention_scores=queries@keys.T
        attention_weights=torch.softmax(attention_scores/keys.shape[-1]**0.5,dim=-1)
        context_vectors=attention_weights@values
        return context_vectors
        
        



In [ ]:

#  Just used nn.Linear for better activation. 



class SelfAttention_v2(nn.Module):
    def __init__(self,d_in,d_out,qkv_bias=False):
        super.__init__()
        self.W_query=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_key=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_value=nn.Linear(d_in,d_out,bias=qkv_bias)
    def forward(self,X):
        queries=x@self.W_query
        keys=x@self.W_key
        values=x@self.W_values
        
        attention_scores=queries@keys.T
        attention_weight=torch.softmax(attention_scores/keys.shape[-1]**0.5,dim=-1)
        context_vectors=attention_weight@values
        return context_vectors
        

In [59]:
class CausalAttention(nn.Module):
    def __init__(self,d_in,d_out,qkv_bias=False):
        super.__init__()
        self.W_query=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_key=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_value=nn.Linear(d_in,d_out,bias=qkv_bias)
    
    def forward(self,x):
        queries=x@self.W_query
        keys=x@self.W_key
        values=x@self.W_value
        
        attention_scores=queries@keys.T
        mask=torch.ones(attention_scores.shape[0],attention_scores.shape[1])
        mask=torch.triu(mask)
        attention_scores[mask==1]=torch.tensor(-float('inf'))
        attention_weight=torch.softmax(attention_scores/keys.shape[-1]**0.5,dim=-1)
        # drop out here
        # The 'train' parameter specifies whether dropout should behave in training mode (drop values) or evaluation mode (no dropout).
        # In PyTorch, when train=True, dropout is applied; when train=False, dropout is bypassed.
        attention_weight = torch.dropout(attention_weight, p=0.5, train=self.training)
        context_vectors=attention_weight@values
        return context_vectors

In [60]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self,d_in,d_out,context_length,drop_out,num_heads,qkv_bias=False):
        super().__init__()
        self.heads=nn.ModuleList([CausalAttention(d_in,d_out,context_length,drop_out,qkv_bias)for _ in range(num_heads)])
    def forward(self,x):
        return torch.cat([head(x) for head in self.heads],dim=-1)
        

In [62]:
class MultiHeadAttention(nn.Module):
    def __init__(self,d_in,d_out,context_length,drop_out,num_heads,qkv_bias=False):
        super.__init__()
        assert d_out % num_heads==0, "d_out must be divisible by num_heads"
        self.W_query=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_key=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_value=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.head_dim=d_out//num_heads
        self.drop_out=nn.Dropout(drop_out)
        self.out_proj=nn.Linear(d_out,d_out)
        self.num_heads=num_heads
        self.register_buffer("mask",torch.triu(torch.ones(context_length,context_length),diagonal=1))
        
        
    def forward(self,x):
        b,num_of_token,d_in=x.shape
        keys=self.W_key(x)
        values=self.W_value(x)
        queries=self.W_query(x)
        
        keys=keys.view(b,num_of_token,self.num_heads,self.head_dim)
        queries=queries.view(b,num_of_token,self.num_heads,self.head_dim)
        values=values.view(b,num_of_token,self.num_heads,self.head_dim)
        
        
        keys.transpose(1,2)
        values.transpose(1,2)
        queries.transpose(1,2)
        
        attention_score=queries@keys.tranpose(2,3)
        mask_bool=self.mask.bool()[:num_of_token,:num_of_token]
        attention_score.fill(mask_bool,-torch.inf)
        
        attention_weights=torch.softmax(attention_score/keys.shape[-1]**2,dim=-1)
        attention_weights=self.drop_out(attention_weights)
        
        context_vector=(attention_weights@values).transpose(1,2)
        
        context_vector=context_vector.contiguous().view(b,num_of_token,self.d_out)
        
        context_vector=self.out_proj(context_vector)
        
        return context_vector
        
        
        
        
        
        

        
    
    
    
    

In [ ]:
for i,x_i in enumerate(input_embeddings[0]):
    print(x_i.shape)
    context_vectors[i]=attention_weight_using_softmax[i]*x_i

torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])


TypeError: len() of a 0-d tensor

In [ ]:
attention_scores[0][1]

tensor([ 46.8761, 494.9998,  -4.3935,  38.6101], grad_fn=<SelectBackward0>)

In [ ]:
input_embeddings[0][0].shape


torch.Size([256])

In [ ]:
attention_weights=attention_scores/attention_scores.sum()

In [ ]:
attention_weights

tensor([ 0.0814,  0.8592, -0.0076,  0.0670], grad_fn=<DivBackward0>)

In [ ]:
attention_weights.sum()

tensor(1., grad_fn=<SumBackward0>)

In [ ]:
def soft_max_naive(x):
    return torch.exp(x)/torch.exp(x).sum(dim=0)

In [ ]:
# The output tensor([0., nan, 0., 0.], grad_fn=<DivBackward0>) happened because the denominator in soft_max_naive was zero or very close to zero.
# Given your attention_scores are tensor([ 46.8761, 494.9998,  -4.3935,  38.6101]), torch.exp(attention_scores) will be extremely large for 494.9998,
# and almost zero for the others, causing the sum to be dominated by one value and the rest to underflow.
# This can cause numerical instability and nan values.
# To fix this, use a numerically stable softmax:
def soft_max_stable(x):
    x_exp = torch.exp(x - x.max())
    return x_exp / x_exp.sum(dim=0)

attention_weight_using_softmax = soft_max_stable(attention_scores)

In [ ]:
print(attention_weight_using_softmax)

tensor([0., 1., 0., 0.], grad_fn=<DivBackward0>)


In [ ]:
# We need dim=0 here because attention_scores is a 1D tensor (a vector), and softmax needs to know along which dimension to compute the softmax.
# For a vector, dim=0 means we apply softmax over the entire vector.
torch.softmax(attention_scores, dim=0)

tensor([0., 1., 0., 0.], grad_fn=<SoftmaxBackward0>)

In [ ]:
query=input_embeddings[0]
print(query)

tensor([[ 0.1470,  1.2117,  1.5106,  ...,  2.7506, -0.8562,  0.0327],
        [-0.8444,  1.3840,  0.2867,  ...,  1.0948,  0.3536, -2.5692],
        [-2.0449, -2.3660, -1.3700,  ...,  2.1413,  0.9736, -0.2204],
        [-0.9697, -1.1252, -1.6693,  ..., -1.6593,  0.7941,  1.6137]],
       grad_fn=<SelectBackward0>)


In [ ]:
context_vectors=torch.zeros(query.shape)

In [ ]:
context_vectors.shape

torch.Size([4, 256])

In [ ]:
for i,x_i in enumerate(input_embeddings[0]):
    print(x_i.shape)
    context_vectors[i]=attention_weight_using_softmax[i]*x_i

torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])


In [ ]:
context_vectors

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.0000,  0.0000],
        [-0.8444,  1.3840,  0.2867,  ...,  1.0948,  0.3536, -2.5692],
        [-0.0000, -0.0000, -0.0000,  ...,  0.0000,  0.0000, -0.0000],
        [-0.0000, -0.0000, -0.0000,  ..., -0.0000,  0.0000,  0.0000]],
       grad_fn=<CopySlices>)